# H2 (Exploratory): Does general AI attitude (GAAIS) moderate
the association between SDT (TENS) and global AI acceptance (UTAUT_AI_mean_imputed), controlling for all confounders?

#### Model structure:
Baseline (main effects): UTAUT = β0 + β1 TENS_c + β2 GAAIS_c + β3 ET_c + β4 PHQ_c + β5 SSRPH_c + β6 age_c + β7 gender + β8 Country + ε

#### Full (interaction):
UTAUT = β0 + β1 TENS_c + β2 GAAIS_c + β3 (TENS_c × GAAIS_c) + same confounders as above + ε

#### confounder-first logic:
- All confounders (GAAIS, ET, symptoms, stigma, age, gender, Country) are controlled in BOTH models.
- Only the SDT × GAAIS term is added in the full model to test moderation

# 0.0 Imports and Path Setup

In [ ]:
from __future__ import annotations

import warnings
from pathlib import Path
from typing import Dict, List

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.anova import anova_lm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from IPython.display import display

warnings.filterwarnings("ignore", category=FutureWarning)
sns.set(style="whitegrid")

PROJECT_ROOT = Path.cwd()
DATA_DIR = PROJECT_ROOT / "data"
OUTPUT_DIR = DATA_DIR / "output"

PROCESSED_PATH = OUTPUT_DIR / "processed_for_analysis.csv"

In [ ]:
print("Unique Country values:")
print(processed["Country"].value_counts(dropna=False))

In [ ]:
print("Unique role_binary values (clinician vs patient cross-country):")
print(processed["role_binary"].value_counts(dropna=False))

In [ ]:
print("Unique role_label_usa3 values (USA-only 3-level role):")
print(processed["role_label_usa3"].value_counts(dropna=False))

# 1.0. Common H2 Definition

## 1.1. Outcomes to analyze

In [ ]:
h2_outcomes = ["Accept_avatar_imputed", "Accept_chatbot_imputed", "Accept_tele_imputed"]

## 1.2. Continuous Variables will be centered in each analytic Sample

In [ ]:
continuous_imputed = [
    "TENS_Life_mean_imputed", "GAAIS_mean_imputed", "ET_mean_imputed", 
    "PHQ5_mean_imputed", "SSRPH_mean_imputed", "age_imputed"
    ]

## 1.3. Helper to fit Baseline main-effects model: SDT + role + confounders and Role-moderation model: SDT * role + same confounders
- For Clean ANOVA comparison
- outcome: e.g., 'Accept_chatbot_imputed'
- role: 'role_binary' or 'role_label_usa3'
- include_country: True when using China+USA; False for USA-only
- label: string for printing (e.g., 'H2a' / 'H2b')

In [ ]:
def fit_role_moderation_for_outcome(outcome: str, df: pd.DataFrame, role_var: str, include_country: bool, label: str):
    
    cols = [
        outcome,
        "TENS_Life_mean_imputed_c",
        "GAAIS_mean_imputed_c",
        "ET_mean_imputed_c",
        "PHQ5_mean_imputed_c",
        "SSRPH_mean_imputed_c",
        "age_imputed_c",
        "gender",
        role_var,
    ]
    if include_country:
        cols.append("Country")

    sub_df = df[cols].dropna().copy()
    if sub_df.empty:
        print(f"[{label}] {outcome}: no complete cases available.")
        return None, None, None

    print(f"{label} – Role moderation for {outcome} (N={len(sub_df)})")

    # Baseline model: SDT + role + confounders
    baseline_formula = (
        f"{outcome} ~ "
        "TENS_Life_mean_imputed_c "
        "+ GAAIS_mean_imputed_c "
        "+ ET_mean_imputed_c "
        "+ PHQ5_mean_imputed_c "
        "+ SSRPH_mean_imputed_c "
        "+ age_imputed_c "
        "+ C(gender) "
        f"+ C({role_var})"
    )
    if include_country:
        baseline_formula += " + C(Country)"

    baseline_model = smf.ols(formula=baseline_formula, data=sub_df).fit()
    print("Baseline model (SDT + role + confounders):")
    display(baseline_model.summary().tables[1])
    print(f"R² (baseline) = {baseline_model.rsquared:.3f}")

    # Role-moderation model: SDT * role + confounders
    role_formula = (
        f"{outcome} ~ "
        f"TENS_Life_mean_imputed_c * C({role_var}) "
        "+ GAAIS_mean_imputed_c "
        "+ ET_mean_imputed_c "
        "+ PHQ5_mean_imputed_c "
        "+ SSRPH_mean_imputed_c "
        "+ age_imputed_c "
        "+ C(gender)"
    )
    if include_country:
        role_formula += " + C(Country)"

    role_model = smf.ols(formula=role_formula, data=sub_df).fit()
    print("Role-moderation model (SDT × role):")
    display(role_model.summary().tables[1])
    print(f"R² (role-moderation) = {role_model.rsquared:.3f}")

    # Model comparison via ANOVA
    print("Model comparison (Baseline vs Role-moderation):")
    comp = anova_lm(baseline_model, role_model)
    display(comp)

    return sub_df, baseline_model, role_model

# 2.0. H2a: Combined China + USA, clinican vs. patient

## 2.1. Build combined analytic sample with clinician vs patient

In [ ]:
h2a_vars = (h2_outcomes + continuous_imputed + ["gender", "Country", "role_binary"])
h2a_df = processed[h2a_vars].copy()

In [ ]:
# Keep only China + USA
h2a_df = h2a_df[h2a_df["Country"].isin(["China", "USA"])].copy()

# Keep only clear clinician vs patient (role_binary) cases
h2a_df = h2a_df[h2a_df["role_binary"].isin(["clinician", "patient"])].copy()

## 2.2. Drop missing gender / country to match confounder sets

In [ ]:
h2a_df = h2a_df.dropna(subset=["gender", "Country", "role_binary"])

In [ ]:
print("H2a analytic sample size:", len(h2a_df))
print("H2a Country distribution:")
print(h2a_df["Country"].value_counts(dropna=False))

In [ ]:
print("H2a role_binary distribution (clinician vs patient):")
print(h2a_df["role_binary"].value_counts(dropna=False))

## 2.3. Center all continuous imputed predictors in combined sample

In [ ]:
for col in continuous_imputed:
    mean_val = h2a_df[col].mean()
    h2a_df[f"{col}_c"] = h2a_df[col] - mean_val
    print(f"H2a – {col} mean for centering: {mean_val:.3f}")

In [ ]:
print("H2a – means of centered variables:")
display(h2a_df[[f"{c}_c" for c in continuous_imputed]].mean())

## 2.4. Fit H2a models for each intervention-specific outcome

In [ ]:
h2a_models: Dict[str, Dict[str, object]] = {}

for outcome in h2_outcomes:
    sub_df, base_m, role_m = fit_role_moderation_for_outcome(
        outcome=outcome,
        df=h2a_df,
        role_var="role_binary",
        include_country=True,
        label="H2a (China & USA, clinician vs patient)",
    )
    h2a_models[outcome] = {
        "data": sub_df,
        "baseline": base_m,
        "role_model": role_m,
    }

## 2.5. Summary table for SDT × Role (clinician vs patient) interaction

In [ ]:
h2a_summary_rows = []

for outcome in h2_outcomes:
    role_model = h2a_models[outcome]["role_model"]
    if role_model is None:
        continue

    # Find the interaction term for SDT × role_binary (whatever the non-reference level is)
    int_terms = [
        p for p in role_model.params.index
        if p.startswith("TENS_Life_mean_imputed_c:C(role_binary)")
    ]

    # 'TENS_Life_mean_imputed_c:C(role_binary)[T.patient]'
    term_name = int_terms[0]
    role_level = term_name.split("[T.")[-1].rstrip("]")

    beta = role_model.params[term_name]
    se = role_model.bse[term_name]
    p = role_model.pvalues[term_name]
    ci_low, ci_high = role_model.conf_int().loc[term_name]
    r2 = role_model.rsquared

    h2a_summary_rows.append({
        "Outcome": outcome,
        "Interaction_level": role_level,
        "beta_TENSxRole(diff_vs_clinician)": beta,
        "SE": se,
        "p": p,
        "CI_low": ci_low,
        "CI_high": ci_high,
        "R2_role_model": r2,
    })

h2a_summary_df = pd.DataFrame(h2a_summary_rows)
print("H2a: SDT × Role (clinician vs patient) interaction summary (China + USA):")
display(h2a_summary_df)

# 3.0 H2b – USA-only, clinician vs patient vs community

In [ ]:
h2b_vars = (h2_outcomes + continuous_imputed + ["gender", "Country", "role_label_usa3"])
h2b_df = processed[h2b_vars].copy()

## 3.1. Restrict to USA only

In [ ]:
h2b_df = h2b_df[h2b_df["Country"] == "USA"].copy()

In [ ]:
# Keep non-missing role_label_usa3 + gender
h2b_df = h2b_df.dropna(subset=["gender", "role_label_usa3"])

In [ ]:
print("H2b analytic sample size (USA-only):", len(h2b_df))
print("H2b role_label_usa3 distribution:")
print(h2b_df["role_label_usa3"].value_counts(dropna=False))

In [ ]:
# Center continuous variables within USA-only sample
for col in continuous_imputed:
    mean_val = h2b_df[col].mean()
    h2b_df[f"{col}_c"] = h2b_df[col] - mean_val
    print(f"H2b – {col} mean for centering (USA-only): {mean_val:.3f}")

In [ ]:
print("H2b – means of centered variables (should be ≈ 0):")
display(h2b_df[[f"{c}_c" for c in continuous_imputed]].mean())

In [ ]:
# Fit H2b models for each outcome (Country is constant, so not included)
h2b_models: Dict[str, Dict[str, object]] = {}
for outcome in h2_outcomes:
    sub_df, base_m, role_m = fit_role_moderation_for_outcome(
        outcome=outcome,
        df=h2b_df,
        role_var="role_label_usa3",
        include_country=False,
        label="H2b (USA-only, clinician vs patient vs community)",
    )
    h2b_models[outcome] = {
        "data": sub_df,
        "baseline": base_m,
        "role_model": role_m,
    }

In [ ]:
# Summary table – SDT × each USA role (vs reference, likely 'patient')
h2b_summary_rows = []
for outcome in h2_outcomes:
    role_model = h2b_models[outcome]["role_model"]
    if role_model is None:
        continue

    print(f"\n[H2b] SDT × role interaction terms for {outcome}:")
    int_terms = [p for p in role_model.params.index
                 if "TENS_Life_mean_imputed_c:C(role_label_usa3)" in p]
    print(int_terms)

    for term in int_terms:
        beta = role_model.params[term]
        se = role_model.bse[term]
        p = role_model.pvalues[term]
        ci_low, ci_high = role_model.conf_int().loc[term]
        r2 = role_model.rsquared

        h2b_summary_rows.append({
            "Outcome": outcome,
            "Interaction_term": term,
            "beta": beta,
            "SE": se,
            "p": p,
            "CI_low": ci_low,
            "CI_high": ci_high,
            "R2_role_model": r2,
        })

h2b_summary_df = pd.DataFrame(h2b_summary_rows)
print("H2b: SDT × Role (role_label_usa3) interaction summary (USA-only):")
display(h2b_summary_df)

## 1.1. Keep rows with non-missing categorical covariates

In [ ]:
n_total = len(h2_df)
h2_df = h2_df.dropna(subset=["gender", "Country"])
n_analytic = len(h2_df)

In [ ]:
print("H2 analytic sample:")
print(f"Total N in processed: {n_total}")
print(f"N with non-missing gender & Country: {n_analytic}")

In [ ]:
print("Country distribution (H2 sample):")
print(h2_df["Country"].value_counts(dropna=False))

In [ ]:
print("Gender distribution (H2 sample):")
print(h2_df["gender"].value_counts(dropna=False))

# 2.0 Descriptive Statistics and Correlations (H2)

We describe the continuous variables and inspect basic correlations among SDT, GAAIS, age, and global AI acceptance.

In [ ]:
continuous_h2 = [
    "UTAUT_AI_mean_imputed",
    "TENS_Life_mean_imputed",
    "GAAIS_mean_imputed",
    "ET_mean_imputed",
    "PHQ5_mean_imputed",
    "SSRPH_mean_imputed",
    "age_imputed",
]

print("Descriptive statistics (H2 continuous variables):")
display(h2_df[continuous_h2].describe().T)

In [ ]:
# Correlation matrix
corr_h2 = h2_df[continuous_h2].corr()
print("Correlation matrix (H2):")
display(corr_h2.round(3))

# 3.0 Center Continuous Predictors

We mean-center SDT (TENS), general AI attitudes (GAAIS), and age for interpretability and to align with moderation conventions.

In [ ]:
center_cols_h2 = [
    "TENS_Life_mean_imputed",
    "GAAIS_mean_imputed",
    "ET_mean_imputed",
    "PHQ5_mean_imputed",
    "SSRPH_mean_imputed",
    "age_imputed",
]

for col in center_cols_h2:
    mean_val = h2_df[col].mean()
    h2_df[f"{col}_c"] = h2_df[col] - mean_val
    print(f"{col} mean for centering: {mean_val:.3f}")

In [ ]:
print("Means of centered variables (should be ≈ 0):")
print(h2_df[[f"{c}_c" for c in center_cols_h2]].mean())

# 4.0 Baseline H2 Model – Main Effects Only

We first estimate a main-effects model without the interaction:

UTAUT_AI = β_0 + β_1 TENS_c + β_2 GAAIS_c + β_3 age_c + β_4 gender + β_5 Country + 𝜀

In [ ]:
baseline_formula_h2 = (
    "UTAUT_AI_mean_imputed ~ "
    "TENS_Life_mean_imputed_c "
    "+ GAAIS_mean_imputed_c "
    "+ ET_mean_imputed_c "
    "+ PHQ5_mean_imputed_c "
    "+ SSRPH_mean_imputed_c "
    "+ age_imputed_c "
    "+ C(gender) + C(Country)"
)

h2_main_model = smf.ols(formula=baseline_formula_h2, data=h2_df).fit()

print("H2 Baseline (main effects only) model summary:")
display(h2_main_model.summary())

In [ ]:
print(f"R² (H2 main-effects model): {h2_main_model.rsquared:.3f}")

In [ ]:
print(f"Adj. R² (H2 main-effects model): {h2_main_model.rsquared_adj:.3f}")

# 5.0. Full H2 Model – SDT × GAAIS Interaction

We add the interaction between SDT and general AI attitudes:

UTAUT_AI = β_0 + β_1 TENS_c + β_2 GAAIS_c + β_3 (TENS_c × GAAIS_c) + covariates + 𝜀

In [ ]:
# Full H2 model with interaction term
full_formula_h2 = (
    "UTAUT_AI_mean_imputed ~ "
    "TENS_Life_mean_imputed_c * GAAIS_mean_imputed_c "
    "+ ET_mean_imputed_c "
    "+ PHQ5_mean_imputed_c "
    "+ SSRPH_mean_imputed_c "
    "+ age_imputed_c "
    "+ C(gender) + C(Country)"
)

h2_full_model = smf.ols(formula=full_formula_h2, data=h2_df).fit()

print("H2 Full model (with SDT × GAAIS interaction) summary:")
h2_full_model.summary()

In [ ]:
print(f"R² (H2 full model): {h2_full_model.rsquared:.3f}")

In [ ]:
print(f"Adj. R² (H2 full model): {h2_full_model.rsquared_adj:.3f}")

## 5.1. Extract Key Composite

In [ ]:
print("Key H2 coefficients (SDT, GAAIS, SDT×GAAIS):")
coef_names = [
    "TENS_Life_mean_imputed_c",
    "GAAIS_mean_imputed_c",
    "TENS_Life_mean_imputed_c:GAAIS_mean_imputed_c",
]
display(h2_full_model.params[coef_names])

In [ ]:
print("Key H2 p-values:")
display(h2_full_model.pvalues[coef_names])

# 6.0. Model Comparison: Main Effects VS Interaction

We compare the main-effects model vs. the interaction model using ANOVA and ΔR².

In [ ]:
print("ANOVA comparison: main-effects vs interaction model (H2)")
anova_results = anova_lm(h2_main_model, h2_full_model)
display(anova_results)

Adding the interaction term reduced the residual error (SSR dropped from 3095.81 → 3091.40), But this reduction is not statistically significant:
- F = 3.17
- p = 0.075 (above .05 but suggestive)

In [ ]:
r2_main = h2_main_model.rsquared
r2_full = h2_full_model.rsquared
delta_r2 = r2_full - r2_main

print(f"R² (main effects): {r2_main:.3f}")
print(f"R² (full with interaction): {r2_full:.3f}")
print(f"ΔR² due to SDT × GAAIS interaction: {delta_r2:.3f}")

# 7.0. Multicollinearity Check
- Build design matrix for VIF (excluding intercept)
- Use the same structure as the full H2 model

In [ ]:
X_h2 = h2_full_model.model.exog
vif_data_h2 = []

for i, name in enumerate(h2_full_model.model.exog_names):
    if name == "Intercept":
        continue
    vif = variance_inflation_factor(X_h2, i)
    vif_data_h2.append({"Predictor": name, "VIF": vif})

vif_h2_df = pd.DataFrame(vif_data_h2).sort_values("VIF", ascending=False)

print("Variance Inflation Factors (H2 full model):")
display(vif_h2_df)

# 8.0. Residual Diagnostics (H2 Full Model)

We check linearity, homoscedasticity, and residual distribution.

In [ ]:
h2_df["resid_h2"] = h2_full_model.resid
h2_df["fitted_h2"] = h2_full_model.fittedvalues

fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# Residuals vs fitted
sns.scatterplot(
    x="fitted_h2",
    y="resid_h2",
    data=h2_df,
    ax=axes[0],
    alpha=0.5
)
axes[0].axhline(0, color="black", linestyle="--", linewidth=1)
axes[0].set_xlabel("Fitted values")
axes[0].set_ylabel("Residuals")
axes[0].set_title("H2 (SDT × GAAIS): Residuals vs Fitted")

# Residual distribution
sns.histplot(h2_df["resid_h2"], kde=True, ax=axes[1])
axes[1].set_xlabel("Residual")
axes[1].set_title("H2 (SDT × GAAIS): Residual Distribution")

plt.tight_layout()
plt.show()

# 9.0. Simple Slopes / Plot-Ready Data (Low / Mean / High GAAIS)

Even if the interaction is non-significant, reviewers often like seeing simple slopes for interpretability. Here we generate predicted UTAUT scores across SDT levels at low / mean / high GAAIS (±1 SD).

In [ ]:
g_mean = h2_df["GAAIS_mean_imputed_c"].mean()   # ~0
g_sd   = h2_df["GAAIS_mean_imputed_c"].std()

g_levels = {
    "low_GAAIS":  g_mean - g_sd,
    "mean_GAAIS": g_mean,
    "high_GAAIS": g_mean + g_sd,
}

# TENS grid (centered)
tens_min = h2_df["TENS_Life_mean_imputed_c"].min()
tens_max = h2_df["TENS_Life_mean_imputed_c"].max()
tens_grid_c = np.linspace(tens_min, tens_max, 50)

pred_rows = []
gender_ref = h2_df["gender"].mode()[0]
country_ref = h2_df["Country"].mode()[0]

for level_name, g_val in g_levels.items():
    for t_val in tens_grid_c:
        row = {
            "TENS_Life_mean_imputed_c": t_val,
            "GAAIS_mean_imputed_c":    g_val,
            "ET_mean_imputed_c":       0.0,
            "PHQ5_mean_imputed_c":     0.0,
            "SSRPH_mean_imputed_c":    0.0,
            "age_imputed_c":           0.0,  # mean-centered age
            "gender":                  gender_ref,
            "Country":                 country_ref,
            "GAAIS_level":             level_name,
        }
        pred_rows.append(row)

pred_df = pd.DataFrame(pred_rows)

# Predicted UTAUT using full model
pred_df["UTAUT_pred"] = h2_full_model.predict(pred_df)

# Back-transform TENS to raw scale for plotting
tens_raw_mean = h2_df["TENS_Life_mean_imputed"].mean()
pred_df["TENS_Life_raw"] = pred_df["TENS_Life_mean_imputed_c"] + tens_raw_mean

print("Preview of simple slopes prediction data (H2):")
display(pred_df.head())

# 12.0. Plot simple slopes of TENS → UTAUT at different GAAIS levels

In [ ]:
plt.figure(figsize=(8, 6))
sns.lineplot(
    data=pred_df,
    x="TENS_Life_raw",
    y="UTAUT_pred",
    hue="GAAIS_level"
)
plt.xlabel("Self-Determination (TENS_Life_mean, raw scale)")
plt.ylabel("Predicted Global AI Acceptance (UTAUT_AI_mean)")
plt.title("H2 (Exploratory): Predicted AI Acceptance Across SDT\n"
          "at Low / Mean / High General AI Attitudes")
plt.tight_layout()
plt.show()

# Narrative Summary

In [ ]:
beta_sdt   = h2_full_model.params["TENS_Life_mean_imputed_c"]
p_sdt      = h2_full_model.pvalues["TENS_Life_mean_imputed_c"]

beta_gaais = h2_full_model.params["GAAIS_mean_imputed_c"]
p_gaais    = h2_full_model.pvalues["GAAIS_mean_imputed_c"]

beta_int   = h2_full_model.params["TENS_Life_mean_imputed_c:GAAIS_mean_imputed_c"]
p_int      = h2_full_model.pvalues["TENS_Life_mean_imputed_c:GAAIS_mean_imputed_c"]

print(
    f"In the exploratory H2 model, higher self-determination (SDT; TENS) was associated with "
    f"greater global acceptance of AI mental-health interventions "
    f"(β = {beta_sdt:.3f}, p = {p_sdt:.3g}), controlling for general AI attitudes (GAAIS), "
    f"epistemic trust, symptoms, stigma, age, gender, and country.\n"
    f"General AI attitudes (GAAIS) also showed a positive association with global AI acceptance "
    f"(β = {beta_gaais:.3f}, p = {p_gaais:.3g}).\n"
    f"The SDT × GAAIS interaction term was "
    f"{'statistically significant' if p_int < 0.05 else 'not statistically significant'} "
    f"(β = {beta_int:.3f}, p = {p_int:.3g}), and the inclusion of the interaction changed R² by "
    f"ΔR² = {delta_r2:.3f} relative to the main-effects model."
)